# Test Functions - Exact Frank Wolfe
This code is to test functions in the `fw_fun.jl` file that we will use for *exact* Frank Wolfe algorithm. We are going to run on one small example, only to see that the code should be correct (bug-free). We will also spend a little bit of time to demonstrate why the code is written as such.

In [1]:
using Revise
includet("fw_fun.jl")

## Test 1: Vertex Cover Gradient Oracle

First, we'll check that the gradient oracle for the undirected weighted vertex cover is correct. Let's first review do this by re-deriving the gradient itself. We'll actually do this for the more general weighted set cover function.

Let $\mathcal{U} = \{u_1, \dots u_m \}$ be a universe of $m$ elements, each with a non-negative weight $w_i$ for $i=1, \dots , m$. Let $\Omega = \{ S_1, \dots S_n \}$ be a collection of subsets $S_i \subset \mathcal{U}$. For a collection 
of subsets, $X \subset \Omega$, define the *weighted set coverage function* to be 
$$f(X) = w \left( \cup_{S_i \in X} S_i \right) \enspace.$$
The multilinear extension $F: [0,1]^n \rightarrow \mathbb{R}$ is given by 
$$F(x) = \sum_{j=1}^m w_j \left( 1 - \prod_{i : u_j \in S_i} (1 - x_i) \right) \enspace. $$
Let's compute the gradient. We'll do this by computing the coordiantes of the gradient, i.e. the partial derivative $\frac{\partial F(x)}{\partial x_k}$ for each coordinate $k=1, \dots n$. Now, by linearity, computing this partial derivative only requires observing that
$$ \frac{\partial }{\partial x_k} \left[ 1 - \prod_{i : u_j \in S_i} (1 - x_i) \right]
= \left\{
\begin{array}{lr}
       \prod_{\substack{i : u_j \in S_i \\ i \neq k}} (1 - x_i) & u_j \in S_k\\
       0 & u_j \notin S_k                     
     \end{array}
\right.$$
Now, multiplying by $w_j$ and summing these partial derivatives up over $j=1 \dots m$ gives us the partial derviative $\frac{\partial F(x)}{\partial x_k}$ that we were interested in.

We are particularly interested in the case of vertex cover, where both the universe and the sets corespond to vertices of a graph and we saw that a vertex "covers" the neighbors that it points to.
Once can check that the function `vertex_cover_grad()` takes as input an adjacency list and (optional) vector of weights and creates a function which returns the exact gradient. It is efficient because it uses the adjacency list to reduce a naive $O(n^3)$ computation to something which looks more like $O(n d^2)$ where $d$ is the average degree, which can be much smaller than $n$.


Let's stary by creating a simple graph, say, a 4-cycle.

In [2]:
adj_dict = Dict(1 => [4,2], 2 => [1,3], 3 => [2,4], 4 => [3,1])

Dict{Int64,Array{Int64,1}} with 4 entries:
  4 => [3, 1]
  2 => [1, 3]
  3 => [2, 4]
  1 => [4, 2]

Now, we'll instantiate a gradient oracle without weights.

In [3]:
f_grad = vertex_cover_grad(4, adj_dict)

f_grad (generic function with 1 method)

Let's check it in a couple of places. First, we know that $\nabla F(0) = 3 \mathbf{1}$ because $f(\{e\}) = 3$ for every vertex $e$. Additionally, we should get that $\nabla F(\mathbf{1}) = 0$.

In [4]:
x = zeros(4)
f_grad(x)

4-element Array{Float64,1}:
 3.0
 3.0
 3.0
 3.0

In [5]:
x = ones(4)
f_grad(x)

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [6]:
f_grad(0.3 * ones(4))

4-element Array{Float64,1}:
 1.89
 1.89
 1.89
 1.89

This looks correct. Now let's see whether using the `weights` arguement behaves well.

In [7]:
weights = [1,2,3,4]
f_grad_w = vertex_cover_grad(4, adj_dict, weights=weights)

f_grad (generic function with 1 method)

In [8]:
f_grad_w(zeros(4))

4-element Array{Float64,1}:
 7.0
 6.0
 9.0
 8.0

In [9]:
f_grad(ones(4))

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Yup, I think that this looks correct. If we wanted to be *extra* sure, we could explicitly compute the gradient $\nabla F(x)$ by hand for a given value of $x$ and then check that this matches what the algorithm returns. I really only checked two corner cases.

## Test 2: Uniform Matroid - Linear Optimization Oracle
This one should be easy. Let's just check it in a few cases.

In [10]:
k = 3
mat_lin_opt = uniform_mat_lin_opt(k)

mat_lin_opt (generic function with 1 method)

In [11]:
c = rand(5)

5-element Array{Float64,1}:
 0.26011109480357897
 0.6687946207031705 
 0.8046414018826553 
 0.501777201585758  
 0.6580127701406866 

In [12]:
S = mat_lin_opt(c)

Set([2, 3, 5])

In [13]:
c = [1,2,3,4,5]
S = mat_lin_opt(c)

Set([4, 3, 5])

Yup, this looks correct.

## Test 3: Partition Matroid - Linear Optimization Oracle
You should recall the definition of partition matroid and convince yourself that the greedy algorithm for linear optimization can work with each partition independently. Then, observe that this is what my code does.

Now let's construct a partition matroid which is relatively simple. We will use the sample ground set (the vertices of the 4-cycle) and our partition will be $P_1 = \{1,2\}$ and $P_2 = \{3,4\}$ where we allow $1$ element from $P_1$ and $2$ elements from $P_2$. Let's create the corresponding partition dictionary.

In [14]:
partition_dict = Dict(1 => (1, [1,2]), 2 => (2, [3,4]))

Dict{Int64,Tuple{Int64,Array{Int64,1}}} with 2 entries:
  2 => (2, [3, 4])
  1 => (1, [1, 2])

and now creating the linear optimization oracle,

In [26]:
part_mat_lin_opt = partition_mat_lin_opt(partition_dict)

(::getfield(Main, Symbol("#mat_lin_opt#10")){Dict{Int64,Tuple{Int64,Array{Int64,1}}}}) (generic function with 1 method)

and choosing a linear function to optimize, 

In [22]:
c = rand(4)

4-element Array{Float64,1}:
 0.9440308915213715
 0.647167988450591 
 0.9548338710311961
 0.3291918519288237

In [27]:
S = part_mat_lin_opt(c)

Set([4, 3, 1])

Yup, that looks like it's working, let's try on more example

In [28]:
c = [1,2,3,4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [29]:
S = part_mat_lin_opt(c)

Set([4, 2, 3])

Great, I believe that it is working correctly.

## Test 4: Set to Vector
This test should be easy, because this function is easy. Let's try one example.

In [30]:
S = Set([1,2])
v = set_to_vec(S, 4)

4-element Array{Int64,1}:
 1
 1
 0
 0

## Test 5: Exact Frank Wolfe Algorithm
Because $n=4$ is small, we can find a solution to the constrained problem very easily. This is just a test to see if my code actually *works*, i.e. it doesn't throw any bugs.

In [31]:
# problem
n = 4
adj_dict = Dict(1 => [4,2], 2 => [1,3], 3 => [2,4], 4 => [3,1]) # specify the graph
partition_dict = Dict(1 => (1, [1,2]), 2 => (2, [3,4])) # specify the partition

# construct the oracles
f_grad = vertex_cover_grad(n, adj_dict) # the gradient oracle for unweighted vertex cover
mat_lin_opt = partition_mat_lin_opt(partition_dict) # the linear optimzation oracle for partition matroid

# run frank wolfe
T = 10 # number of iterations (inverse step size)
x, v_set = fw_exact(n, T, f_grad, mat_lin_opt)

([1.0, 0.0, 1.0, 1.0], Any[Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1]), Set([4, 3, 1])])

Yes, everything seemed to work fine. Plus, this is a small enough example that we can actually verify what the algorithm will do at every step. Here, I believe that picking the same base every time was what it should have done, although we might want to analytically verify this.

# Next Steps
There are several next steps.

1. **More Oracles** Implement more linear optimization oracles for specific matroids and set functions where the multilinear extension admits a closed-form solution. e.g. graphic matroids, realizable matroids
2. **Rounding Schemes** Implement swap rounding algorithm of Chekuri et al for several of our favorite matroids. Note that we can work directly with the bases returned in `v_set`, rather than x. In fact, we could even do the swaps at each iteration of Frank Wolfe, but let's save that for later.
3. **Stochastic FW Variants** There are several stochastic Frank Wolfe variants that we would like to implement. This requires coding (1) stochastic gradient oracle functions (2) 